# Regression models with Keras
I will use Keras Neural Network API to build a regression model.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
# Check that GPU is working
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16846354399530752790, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6662668288
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2355642987061210628
 physical_device_desc: "device: 0, name: GeForce GTX 1070 Ti, pci bus id: 0000:09:00.0, compute capability: 6.1"]

## About the data

The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them.

In [3]:
CSV_PATH = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv'
concrete_data = pd.read_csv(CSV_PATH)
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
concrete_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
Cement                1030 non-null float64
Blast Furnace Slag    1030 non-null float64
Fly Ash               1030 non-null float64
Water                 1030 non-null float64
Superplasticizer      1030 non-null float64
Coarse Aggregate      1030 non-null float64
Fine Aggregate        1030 non-null float64
Age                   1030 non-null int64
Strength              1030 non-null float64
dtypes: float64(8), int64(1)
memory usage: 72.5 KB


In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


The data looks very clean

## Train test split

In [6]:
X = concrete_data.drop('Strength', axis=1)
y = concrete_data['Strength']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

## Preprocessing

In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [8]:
X_train

array([[ 1.82397267,  0.52449106, -0.84805943, ..., -1.54479455,
         0.08715809,  0.1646987 ],
       [ 0.99378155, -0.62989123,  0.61027759, ..., -0.43182836,
         0.8715104 , -0.68661169],
       [-1.25999867, -0.86357185,  1.97708069, ..., -1.71582079,
         1.3569442 , -0.28505019],
       ...,
       [-1.18547015,  1.97564773, -0.84805943, ...,  0.65652043,
        -0.94423433, -0.28505019],
       [-0.6694309 ,  2.82975041, -0.84805943, ...,  0.07347644,
        -1.05169677, -0.28505019],
       [-0.32886386, -0.86357185,  1.09276994, ...,  1.49610377,
         0.32678698,  0.1646987 ]])

In [9]:
y_train

144    72.30
488    22.75
974    15.53
895    49.77
627     7.84
       ...  
688     2.33
621    34.49
850    37.36
583    37.81
332    60.32
Name: Strength, Length: 824, dtype: float64

## Building a neural network

In [10]:
n_columns = X_train.shape[1]
print("Input columns:", n_columns)

Input columns: 8


In [11]:
def regression_model():
    model = Sequential()
    model.add(Dense(50, activation="relu", input_shape=(n_columns,)))
    model.add(Dense(50, activation="relu"))
    model.add(Dense(1))
    
    model.compile(optimizer="adam", loss="mean_squared_error")
    return model

The above function create a model with two hidden layers, each of 50 units

In [12]:
model = regression_model()

In [13]:
model.fit(X_train, y_train, validation_split=0.3, epochs=200, verbose=2)

Train on 576 samples, validate on 248 samples
Epoch 1/200
 - 0s - loss: 1457.6098 - val_loss: 1607.0443
Epoch 2/200
 - 0s - loss: 1356.0769 - val_loss: 1476.2773
Epoch 3/200
 - 0s - loss: 1213.4990 - val_loss: 1282.6541
Epoch 4/200
 - 0s - loss: 1011.0415 - val_loss: 1027.2328
Epoch 5/200
 - 0s - loss: 762.2809 - val_loss: 738.5392
Epoch 6/200
 - 0s - loss: 511.5676 - val_loss: 478.3033
Epoch 7/200
 - 0s - loss: 319.5884 - val_loss: 316.3618
Epoch 8/200
 - 0s - loss: 225.4970 - val_loss: 259.4221
Epoch 9/200
 - 0s - loss: 196.8751 - val_loss: 246.4409
Epoch 10/200
 - 0s - loss: 187.3046 - val_loss: 238.2209
Epoch 11/200
 - 0s - loss: 180.4790 - val_loss: 233.1602
Epoch 12/200
 - 0s - loss: 174.7415 - val_loss: 227.0937
Epoch 13/200
 - 0s - loss: 170.5067 - val_loss: 223.1551
Epoch 14/200
 - 0s - loss: 166.2273 - val_loss: 217.0611
Epoch 15/200
 - 0s - loss: 162.7951 - val_loss: 213.2734
Epoch 16/200
 - 0s - loss: 159.5626 - val_loss: 208.3975
Epoch 17/200
 - 0s - loss: 156.5653 - val_l

Epoch 146/200
 - 0s - loss: 29.5121 - val_loss: 58.5579
Epoch 147/200
 - 0s - loss: 28.8665 - val_loss: 61.2332
Epoch 148/200
 - 0s - loss: 28.6934 - val_loss: 58.5787
Epoch 149/200
 - 0s - loss: 28.5102 - val_loss: 60.1447
Epoch 150/200
 - 0s - loss: 28.1384 - val_loss: 60.4736
Epoch 151/200
 - 0s - loss: 28.2582 - val_loss: 58.7283
Epoch 152/200
 - 0s - loss: 29.0631 - val_loss: 58.0772
Epoch 153/200
 - 0s - loss: 28.1479 - val_loss: 58.6036
Epoch 154/200
 - 0s - loss: 28.2142 - val_loss: 58.0265
Epoch 155/200
 - 0s - loss: 28.0154 - val_loss: 58.3694
Epoch 156/200
 - 0s - loss: 27.6666 - val_loss: 61.4363
Epoch 157/200
 - 0s - loss: 27.5688 - val_loss: 58.9232
Epoch 158/200
 - 0s - loss: 27.5617 - val_loss: 57.2464
Epoch 159/200
 - 0s - loss: 28.4073 - val_loss: 65.7265
Epoch 160/200
 - 0s - loss: 28.2526 - val_loss: 57.8687
Epoch 161/200
 - 0s - loss: 26.9035 - val_loss: 59.5629
Epoch 162/200
 - 0s - loss: 26.8676 - val_loss: 56.6216
Epoch 163/200
 - 0s - loss: 26.5852 - val_loss: 

In [14]:
y_predicted = model.predict(X_test)

In [15]:
mean_squared_error(y_test, y_predicted)

40.74319687808315

Let's compare predicted and original values

In [16]:
for a, b in zip(y_predicted, y_test):
    print(a, b)

[47.632687] 35.3
[24.11264] 21.78
[34.101665] 31.45
[41.4828] 43.58
[55.335983] 59.3
[27.49111] 26.31
[31.059862] 37.36
[40.25158] 35.85
[32.208134] 36.99
[17.54696] 13.52
[44.468086] 41.54
[14.459024] 7.32
[23.591824] 27.04
[45.307434] 50.77
[22.668001] 19.54
[40.208767] 33.19
[41.910984] 24.1
[50.76964] 49.8
[33.959557] 31.97
[38.232452] 43.38
[20.60923] 20.77
[30.576345] 31.42
[47.055862] 33.94
[52.21104] 46.2
[19.673517] 24.13
[41.432404] 32.96
[20.863745] 24.28
[43.481083] 39.42
[37.45876] 42.22
[34.145966] 42.64
[32.605442] 33.76
[47.10385] 52.83
[51.170742] 41.05
[37.247456] 43.06
[31.701513] 25.56
[46.87975] 48.85
[24.53596] 6.27
[38.424137] 42.7
[49.57286] 55.51
[13.570871] 9.45
[48.140385] 50.53
[42.54028] 49.19
[21.397905] 21.16
[13.237145] 13.36
[41.691143] 53.3
[31.49711] 30.88
[15.204544] 14.2
[58.654648] 44.42
[43.72601] 54.38
[47.646698] 44.09
[28.471088] 21.75
[66.49429] 54.77
[22.987978] 16.5
[59.96572] 60.2
[43.984463] 41.1
[32.63973] 26.74
[28.235382] 33.66
[52.6309

Not so precise but that model was fast and dirty.